In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as k

In [2]:
xs = np.array([-1, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

In [3]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [4]:
def create_base_model():
    input = Input(shape=(1,))
    output = Dense(1)(input)
    model = Model(input, output)
    return model

In [5]:
model = create_base_model()
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10.0]))

[[18.986559]]


In [6]:
def huber_loss(y_true, y_pred):
    threshold = 1
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5*threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [7]:
model = create_base_model()
model.compile(optimizer='sgd', loss='huber_loss')
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10.0]))

[[18.540232]]


In [8]:
def huber_loss_with_threshold(threshold):
    def huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5*threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    return huber_loss

In [9]:
model = create_base_model()
model.compile(optimizer='sgd', loss=huber_loss_with_threshold(threshold=1))
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10.0]))

[[18.37589]]


In [10]:
from tensorflow.keras.losses import Loss

class MyHuberLoss(Loss):
    threshold = 1
    def __init__(self, threshold):
        super().__init__()
        self.threshold= threshold
    
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5*self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

In [11]:
model = create_base_model()
model.compile(optimizer='sgd', loss=MyHuberLoss(threshold=1))
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10.0]))

[[18.688143]]
